<a href="https://colab.research.google.com/github/carolynw898/STAT946Proj/blob/main/DiffuSym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
print('Symbolic Regression with Diffusion')

Symbolic Regression with Diffusion


In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import glob
import json
from torch.utils.data import Dataset, DataLoader
import re
import numpy as np
import tqdm
import random
from utils import generateDataStrEq, processDataFiles, CharDataset
from models import PointNetConfig, tNet

In [3]:
blockSize = 32
numVars = 1
numYs = 1
numPoints = 250
target = 'Skeleton'
addVars = True
const_range = [-2.1, 2.1]
trainRange = [-3.0, 3.0]
decimals = 8
embSize = 512


In [4]:
from utils import processDataFiles

# process training files from scratch
path = "0_1_0_13062021_174033.json"
files = glob.glob(path)
text = processDataFiles(files)
chars = sorted(list(set(text))+['_','T','<','>',':']) # extract unique characters from the text before converting the text to a list, # T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
trainText = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(trainText) # shuffle the dataset, it's important specailly for the combined number of variables experiment
train_dataset = CharDataset(text, blockSize, chars, numVars=numVars, 
                numYs=numYs, numPoints=numPoints, target=target, addVars=addVars,
                const_range=const_range, xRange=trainRange, decimals=decimals, augment=False)

data has 967 examples, 52 unique.


In [30]:
train_dataset

In [31]:
loader = DataLoader(train_dataset)

In [32]:
train_dataset.__getitem__(5)

2
250


(tensor([23, 12, 22, 25,  5, 49, 12,  5,  5, 13,  6, 25,  5, 49, 12,  6, 25, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]),
 tensor([12, 22, 25,  5, 49, 12,  5,  5, 13,  6, 25,  5, 49, 12,  6, 25, 24, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]),
 tensor([[ 0.9774,  2.0756,  1.2248,  2.5588, -2.5195, -0.6410,  1.1526, -2.1722,
          -2.0410, -2.7973,  2.5072,  0.9566,  0.4874,  1.1751,  2.3139, -0.8710,
           1.3261, -2.3743, -2.7984,  1.3694,  0.1516, -2.9298,  0.7702, -1.3082,
           2.4207,  0.0958, -2.2913,  0.0600,  1.5200,  2.2456,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0

In [33]:
pnConfig = PointNetConfig(embeddingSize = embSize, numberofPoints = numPoints, numberofVars = numVars, numberofYs = numYs)

In [7]:
pnConfig

In [34]:
tNet_test = tNet(pnConfig)
tNet_test

tNet(
  (conv1): Conv1d(2, 512, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(1024, 2048, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (input_batch_norm): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [35]:
for x, y, p, v in loader:
    print(v)
    break

2
250
tensor([1])


In [ ]:
loader

966

In [36]:
for x, y, p, v in loader:
    pointnet = tNet_test(p)
    print(pointnet)
    break

2
250


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 1024])

In [2]:
from scipy.optimize import minimize
from utils import lossFunc

path = "0_1_0_13062021_174033.json"
files = glob.glob(path)
textTest = processDataFiles(files = files)
textTest = textTest.split('\n')

# this optimizes the constants for a single equation and set of points
# (put it in a loop if you want to optimize multiple instances)
ii = 5 # index of the json of interest
t = json.loads(textTest[ii])

print(t)

{'X': [[0.97741914], [2.07557888], [1.22484007], [2.55880293], [-2.51946864], [-0.64099928], [1.15255076], [-2.1722235], [-2.04097206], [-2.79728418], [2.50716668], [0.95658041], [0.48741062], [1.17511183], [2.31386112], [-0.87096013], [1.32607597], [-2.37433758], [-2.79840294], [1.369396], [0.15157011], [-2.9297556], [0.77019269], [-1.30818162], [2.42074463], [0.09580567], [-2.29125689], [0.05998444], [1.52004739], [2.24561221]], 'Y': [-0.23697606, -5.11652985, -0.90697265, -8.82254165, 8.42045587, -0.33814321, -0.6853927, 5.71671659, 4.82287547, 10.9378401, -8.38106386, -0.19195015, 0.35279748, -0.75225783, -6.82493588, -0.03925367, -1.25311506, 7.23059031, 10.94861417, -1.414008, 0.19122563, 12.24908443, 0.13198265, 1.12408928, -7.66651495, 0.11984972, 6.58812592, 0.0673077, -2.03314716, -6.31194058], 'EQ': '-2.03995581177251*x1**2+1.784589431926912*x1+-0.0323998685508311', 'Skeleton': 'C*x1**2+C*x1+C', 'XT': [[3.23410401], [-3.94730326], [-4.34343974], [4.3425741], [-3.80792522], [

In [3]:
from math import exp, cos, sin

predicted = "C*exp(C*x1)*C*cos(C*x1**2+C*x1)+C" # the predicted equation skeleton with constants denoted with placeholder 'C'

# constants to predict: initialize all to 1
c = [1.0 for i,x in enumerate(predicted) if x=='C']
# apparently the optimizing the constants is the slowest part of the algorithm
cHat = minimize(lossFunc, c, #bounds=b, 
                args=(predicted, t['X'], t['Y'])) 

predicted = predicted.replace('C','{}').format(*cHat.x)

In [4]:
predicted

'-0.1340749819933953*exp(3.2933080234123073*x1)*-0.13407540718076677*cos(-0.6386390933832335*x1**2+3.4320090473687492*x1)+0.17628821986747237'